In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  # noqa
from torch_geometric.nn import global_max_pool
from torch_geometric.data import Batch, Data

import numpy as np

import time

In [2]:
# dataset = 'Cora'
# __file__ = "/app/pytorch_geometric/examples/gcn.py"

# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', dataset)
# dataset = Planetoid(path, dataset, T.NormalizeFeatures())
# data = dataset[0]

# print(data.x.dtype, data.y.dtype, data.edge_index.dtype)

torch.float32 torch.int64 torch.int64


In [3]:
nodes = 18000
hits = 18000
features = 2
edges_per_node = 4

x = torch.zeros([nodes, features], dtype=torch.float32)
# x_mask = torch.zeros([nodes,], dtype=torch.uint8)
y = torch.zeros([1], dtype=torch.int64)
edge_index = torch.zeros([nodes, nodes], dtype=torch.int64)

perm = np.random.permutation(nodes)[:hits]
# x_mask[perm] = 1
x[perm] = torch.randn(hits, features, dtype=torch.float32)

for _ in range(edges_per_node):
    edge_index[np.random.permutation(nodes), np.random.permutation(nodes)] = 1
# for _ in range(edges_per_node):
#     edge_index[np.random.permutation(nodes)[:hits], np.random.permutation(nodes)[:hits]] = 1

# import pickle
# with open("edges_dict.pkl", 'rb') as f:
#     edges = pickle.load(f)
    
#     for k,vs in edges.items():
#         for v in vs:
#             edge_index[k,v] = 1

data = Data(x=x, y=y, edge_index=edge_index.to_sparse()._indices())

print(x.to_sparse())
print(edge_index.to_sparse())

tensor(indices=tensor([[    0,     0,     1,  ..., 17998, 17999, 17999],
                       [    0,     1,     0,  ...,     1,     0,     1]]),
       values=tensor([-1.0178, -0.7190, -1.2644,  ...,  0.6298, -0.8303,
                       1.1015]),
       size=(18000, 2), nnz=36000, layout=torch.sparse_coo)
tensor(indices=tensor([[    0,     0,     0,  ..., 15807, 15807, 15807],
                       [    1,    11,    18,  ..., 15804, 15805, 15806]]),
       values=tensor([1, 1, 1,  ..., 1, 1, 1]),
       size=(18000, 18000), nnz=66552, layout=torch.sparse_coo)


In [4]:
batch = Batch.from_data_list([data for i in range(32)])

print(batch.edge_index.shape)
print(batch.x.to_sparse())

torch.Size([2, 2129664])
tensor(indices=tensor([[     0,      0,      1,  ..., 575998, 575999, 575999],
                       [     0,      1,      0,  ...,      1,      0,      1]]),
       values=tensor([-1.0178, -0.7190, -1.2644,  ...,  0.6298, -0.8303,
                       1.1015]),
       size=(576000, 2), nnz=1152000, layout=torch.sparse_coo)


In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(2, 16, cached=True)
        self.conv2 = GCNConv(16, 16, cached=True)
        self.conv3 = GCNConv(16, 5, cached=True)

#         self.conv1 = ChebConv(2, 16, K=2)
#         self.conv2 = ChebConv(16, 5, K=2)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv3(x, edge_index)
        x = global_max_pool(x, batch)
        return F.log_softmax(x, dim=1)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= Net().to(device)
batch = batch.to(device)
# data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [7]:
def train():
    model.train()
    optimizer.zero_grad()
#     output = model(batch.x.to_sparse(), batch.edge_index, batch.batch)
    output = model(batch.x, batch.edge_index, batch.batch)

    F.nll_loss(output, batch.y).backward()
    optimizer.step()


def test():
    model.eval()
    logits = model(batch.x, batch.edge_index, batch.batch)
    pred = logits.argmax(1)
    acc = pred.eq(batch.y).sum().item() / batch.y.shape[0]
    return acc

In [8]:
start = time.time()

best_acc = 0
batch.x = batch.x.to_sparse()
for epoch in range(1, 201):
    train()
#     train_acc = test()
#     if train_acc > best_acc:
#         best_acc = train_acc
#     log = 'Epoch: {:03d}, Train: {:.4f}, Best: {:.4f}'
#     print(log.format(epoch, train_acc, best_acc))
print(time.time() - start)

25.126508951187134
